<a href="https://colab.research.google.com/github/brunalvess1991/TechLead-2025.1-FerramentasDeIA/blob/main/books/aula_03_model_prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain: Models, Prompts and Output Parsers

## Get your [OpenAI API Key](https://platform.openai.com/account/api-keys)

In [3]:
!pip install python-dotenv
!pip install openai

In [13]:
from IPython.display import display, HTML
display(HTML(
"""
<a target="_blank" href="https://colab.research.google.com/github/pedrodiamel/agents-mini-course/blob/course/books/aula_03_model_prompt.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
"""
))

In [14]:
import os
import openai
from utils import format_message, show_prompt


from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [15]:
# Set the model variable based on the current date
llm_model = "gpt-4o-mini"

## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.

In [16]:
# Ref: https://platform.openai.com/docs/api-reference/chat/create

client = openai.OpenAI()

def get_completion(prompt, model=llm_model):
    messages = [
        {
        "role": "user",
        "content": prompt
        }
    ]

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )

    return response.choices[0].message.content


In [17]:
client = openai.OpenAI()

with client.chat.completions.stream(
    model=llm_model,
    messages=[{"role": "user", "content": "Explique brevemente o que é aprendizado de máquina."}],
    temperature=0,
) as stream:
    for event in stream:
        if event.type == "message.delta":
            print(event.delta, end="", flush=True)

In [18]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [19]:
response = get_completion("Explique brevemente o que é aprendizado de máquina.")
print(response)

Aprendizado de máquina é um subcampo da inteligência artificial que se concentra no desenvolvimento de algoritmos e modelos que permitem que os computadores aprendam a partir de dados. Em vez de serem programados explicitamente para realizar uma tarefa, os sistemas de aprendizado de máquina usam padrões e inferências a partir de dados para melhorar seu desempenho ao longo do tempo. Isso pode incluir tarefas como classificação, regressão, reconhecimento de padrões e tomada de decisões. O aprendizado de máquina é amplamente utilizado em diversas aplicações, como reconhecimento de voz, recomendação de produtos, detecção de fraudes e muito mais.


In [20]:
styles = ["formal and technical", "casual and friendly", "enthusiastic and persuasive", "concise and to the point", "storytelling and engaging"]
tones = ["confident", "empathetic", "urgent", "optimistic", "serious"]

In [21]:
prompt_task = """
Voce é um assistente de IA que ajuda a redigir emails corporativos. Para o contexto, hoje é {date}.

<Task>
Redigir um email com base no contexto fornecido.
</Task>

O email precisa ter a seguinte estrutura básica:
<Structure>
1. Saudação
2. Introdução ao problema
3. Apresentação da solução
4. Benefícios da solução
5. Chamada para ação (CTA)
6. Despedida
</Structure>
"""

prompt_context = """
<Context>
Escreva um email para um cliente corporativo com o seguinte contexto:
{customer_email}
O email deve ser escrito em um dos seguintes estilos: {style} e tons: {tone}.
</Context>
"""

prompt_instructions = """
<Instrutions>
Use o estilo e tom especificados para redigir o email.
Certifique-se de que o email seja claro, profissional e adequado ao público-alvo.
Não se esqueça de incluir um CTA (Call to Action) para agendar uma demonstração.
Não invente informações; baseie-se apenas no contexto fornecido.
Opcionalmente, use gatilhos mentais com autoridade, urgência, pertencimento, benefício se apropriado.
</Instrutions>
"""

prompt_references = """
<Example>
Por exemplo, um email formal e técnico com tom confiante pode ser:
Asunto: Apresentação de Solução Avançada de Visão Computacional para Inspeção Industrial
Prezado Sr. Silva,
Gostaria de apresentar nossa avançada solução de visão computacional projetada para otimizar processos de inspeção industrial.
Nossa tecnologia utiliza algoritmos de ponta para garantir precisão e eficiência, reduzindo custos operacionais.
Ficaria honrado em agendar uma demonstração para discutir como nossa solução pode beneficiar sua empresa.
Atenciosamente,
João Pereira
</Example>
"""

prompt = f"""{prompt_task}
{prompt_context}
{prompt_instructions}
{prompt_references}
"""


In [22]:
show_prompt(prompt)

╭──────────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│  Voce é um assistente de IA que ajuda a redigir emails corporativos. Para o contexto, hoje é {date}.            │
│                                                                                                                 │
│  <Task>                                                                                                         │
│  Redigir um email com base no contexto fornecido.                                                               │
│  </Task>                                                                                                        │
│                                                                                                                 │
│  O email precisa ter a seguinte estrutura básica:                                                               │
│  <Structure>                                                                                                    │
│  1. Saudação                                                                                                    │
│  2. Introdução ao problema                                                                                      │
│  3. Apresentação da solução                                                                                     │
│  4. Benefícios da solução                                                                                       │
│  5. Chamada para ação (CTA)                                                                                     │
│  6. Despedida                                                                                                   │
│  </Structure>                                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
│  <Context>                                                                                                      │
│  Escreva um email para um cliente corporativo com o seguinte contexto:                                          │
│  {customer_email}                                                                                               │
│  O email deve ser escrito em um dos seguintes estilos: {style} e tons: {tone}.                                  │
│  </Context>                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
│  <Instrutions>                                                                                                  │
│  Use o estilo e tom especificados para redigir o email.                                                         │
│  Certifique-se de que o email seja claro, profissional e adequado ao público-alvo.                              │
│  Não se esqueça de incluir um CTA (Call to Action) para agendar uma demonstração.                               │
│  Não invente informações; baseie-se apenas no contexto fornecido.                                               │
│  Opcionalmente, use gatilhos mentais com autoridade, urgência, pertencimento, benefício se apropriado.          │
│  </Instrutions>                                                                                                 │
│                                                       

In [23]:
customer_email = """
Crie um email para um cliente corporativo apresentando nossa solução
de visão computacional para inspeção industrial. O estilo deve ser
formal e técnico, com tom confiante e persuasivo. Inclua um CTA para
agendar uma demonstração.
"""

print(customer_email)


Crie um email para um cliente corporativo apresentando nossa solução
de visão computacional para inspeção industrial. O estilo deve ser
formal e técnico, com tom confiante e persuasivo. Inclua um CTA para
agendar uma demonstração.



In [24]:
from datetime import datetime
def get_today_str() -> str:
    """Get current date in a human-readable format."""
    return datetime.now().strftime("%a %b %-d, %Y")

prompt.format(customer_email=customer_email, style=styles[0], tone=tones[0], date=get_today_str())
response = get_completion(prompt)

In [25]:
print(response)

Assunto: Proposta de Solução para Otimização de Processos

Prezado [Nome do Cliente],

Espero que este email o encontre bem. 

Identificamos que sua empresa enfrenta desafios na otimização de processos, o que pode impactar a eficiência e a produtividade. Sabemos que, em um mercado competitivo, é crucial encontrar maneiras de melhorar continuamente.

Para abordar essa questão, gostaríamos de apresentar nossa solução inovadora de automação de processos. Nossa plataforma foi desenvolvida para integrar-se facilmente aos sistemas existentes, proporcionando uma gestão mais eficiente e reduzindo o tempo gasto em tarefas manuais.

Os benefícios dessa solução incluem a redução de custos operacionais, aumento da produtividade e a possibilidade de realocar recursos para atividades estratégicas. Além disso, nossa equipe de especialistas estará à disposição para garantir uma implementação tranquila e eficaz.

Gostaríamos de convidá-lo para agendar uma demonstração personalizada, onde poderemos most

## Chat API : LangChain

Let's try how we can do the same using LangChain.

In [26]:
!pip install --upgrade langchain
!pip install langchain_openai

### Model

In [27]:
from langchain_openai import ChatOpenAI

In [28]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(
    temperature=0.0,
    model=llm_model
    )
chat

ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x7a13d4b2d5e0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7a13d4b2e450>, root_client=<openai.OpenAI object at 0x7a13d669cfb0>, root_async_client=<openai.AsyncOpenAI object at 0x7a13d4b2cd40>, model_name='gpt-4o-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_organization='org-5qZfsmr4Uz6f4Olo3UqM4Fgb', stream_usage=True)

In [29]:
from langchain_core.messages import HumanMessage

prompt.format(
    customer_email=customer_email,
    style=styles[0],
    tone=tones[0],
    date=get_today_str()
)

response = chat.invoke([HumanMessage(content=prompt)])
print("DONE")

DONE


In [30]:
from utils import format_message, show_prompt

show_prompt(prompt)
format_message([response])

╭──────────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│  Voce é um assistente de IA que ajuda a redigir emails corporativos. Para o contexto, hoje é {date}.            │
│                                                                                                                 │
│  <Task>                                                                                                         │
│  Redigir um email com base no contexto fornecido.                                                               │
│  </Task>                                                                                                        │
│                                                                                                                 │
│  O email precisa ter a seguinte estrutura básica:                                                               │
│  <Structure>                                                                                                    │
│  1. Saudação                                                                                                    │
│  2. Introdução ao problema                                                                                      │
│  3. Apresentação da solução                                                                                     │
│  4. Benefícios da solução                                                                                       │
│  5. Chamada para ação (CTA)                                                                                     │
│  6. Despedida                                                                                                   │
│  </Structure>                                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
│  <Context>                                                                                                      │
│  Escreva um email para um cliente corporativo com o seguinte contexto:                                          │
│  {customer_email}                                                                                               │
│  O email deve ser escrito em um dos seguintes estilos: {style} e tons: {tone}.                                  │
│  </Context>                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
│  <Instrutions>                                                                                                  │
│  Use o estilo e tom especificados para redigir o email.                                                         │
│  Certifique-se de que o email seja claro, profissional e adequado ao público-alvo.                              │
│  Não se esqueça de incluir um CTA (Call to Action) para agendar uma demonstração.                               │
│  Não invente informações; baseie-se apenas no contexto fornecido.                                               │
│  Opcionalmente, use gatilhos mentais com autoridade, urgência, pertencimento, benefício se apropriado.          │
│  </Instrutions>                                                                                                 │
│                                                       

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ Assunto: Proposta de Solução para Otimização de Processos                                                       │
│                                                                                                                 │
│ Prezado [Nome do Cliente],                                                                                      │
│                                                                                                                 │
│ Espero que este email o encontre bem.                                                                           │
│                                                                                                                 │
│ Nos últimos meses, temos observado que muitas empresas enfrentam desafios significativos na otimização de seus  │
│ processos operacionais, resultando em ineficiências e aumento de custos. Compreendemos que a sua organização    │
│ pode estar passando por situações semelhantes.                                                                  │
│                                                                                                                 │
│ Para abordar essa questão, gostaríamos de apresentar nossa solução inovadora de automação de processos, que foi │
│ desenvolvida para simplificar e agilizar as operações diárias. Nossa plataforma utiliza tecnologia de ponta     │
│ para integrar diferentes sistemas, permitindo uma gestão mais eficiente e uma redução significativa no tempo de │
│ execução das tarefas.                                                                                           │
│                                                                                                                 │
│ Os benefícios dessa solução são claros: além de aumentar a produtividade, você poderá reduzir custos            │
│ operacionais e melhorar a qualidade dos serviços prestados. Com a automação, sua equipe poderá focar em         │
│ atividades estratégicas, impulsionando ainda mais o crescimento da sua empresa.                                 │
│                                                                                                                 │
│ Gostaríamos de convidá-lo para agendar uma demonstração personalizada, onde poderemos mostrar como nossa        │
│ solução pode ser implementada em sua organização e os resultados que você pode esperar. Por favor, responda a   │
│ este email com sua disponibilidade, e ficaremos felizes em marcar um horário que seja conveniente para você.    │
│                                                                                                                 │
│ Agradeço pela atenção e estou à disposição para quaisquer dúvidas.                                              │
│                                                                                                                 │
│ Atenciosamente,                                                                                                 │
│                                                                                                                 │
│ [Seu Nome]                                                                                                      │
│ [Seu Cargo]                                                                                                     │
│ [Nome da Empresa]                                                                                               │
│ [Telefone]                                                                                                      │
│ [Email]                                                                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

### Format output

In [31]:
from pydantic import BaseModel, Field

class EmailResponse(BaseModel):
    customer_email: str = Field(..., description="The email content generated for the customer.")
    tips: str = Field(..., description="Additional tips for improving the email.")
    chain_of_thought: str = Field(..., description="The reasoning process behind the email generation.")
    score: int = Field(..., description="A score from 1 to 10 evaluating the quality of the email.")


In [32]:
email_writer_chain = (
    chat
    .with_structured_output(EmailResponse)
)

prompt.format(
    customer_email=customer_email,
    style=styles[0],
    tone=tones[0],
    date=get_today_str()
)

response = email_writer_chain.invoke([HumanMessage(content=prompt)])
print("DONE")

DONE


In [33]:
response

EmailResponse(customer_email='Prezado Sr. Almeida,\n\nEspero que esteja bem. Estou entrando em contato para discutir um desafio que muitas empresas enfrentam atualmente: a necessidade de otimizar a gestão de projetos e aumentar a eficiência das equipes.\n\nPara isso, gostaria de apresentar nossa plataforma de gerenciamento de projetos, que oferece ferramentas integradas para planejamento, acompanhamento e colaboração em tempo real. Com nossa solução, sua equipe poderá visualizar o progresso dos projetos de forma clara e organizada, facilitando a comunicação e a tomada de decisões.\n\nOs benefícios são significativos: aumento da produtividade, redução de retrabalho e melhor alocação de recursos. Além disso, nossa plataforma é intuitiva e pode ser adaptada às necessidades específicas da sua empresa.\n\nGostaria de agendar uma demonstração para que você possa ver em primeira mão como nossa solução pode transformar a gestão de projetos na sua organização. Por favor, me avise qual dia e hor

In [34]:
print("Customer Email:")
print(response.customer_email)
print("\nTips:")
print(response.tips)
print("\nChain of Thought:")
print(response.chain_of_thought)
print("\nScore:")
print(response.score)

Customer Email:
Prezado Sr. Almeida,

Espero que esteja bem. Estou entrando em contato para discutir um desafio que muitas empresas enfrentam atualmente: a necessidade de otimizar a gestão de projetos e aumentar a eficiência das equipes.

Para isso, gostaria de apresentar nossa plataforma de gerenciamento de projetos, que oferece ferramentas integradas para planejamento, acompanhamento e colaboração em tempo real. Com nossa solução, sua equipe poderá visualizar o progresso dos projetos de forma clara e organizada, facilitando a comunicação e a tomada de decisões.

Os benefícios são significativos: aumento da produtividade, redução de retrabalho e melhor alocação de recursos. Além disso, nossa plataforma é intuitiva e pode ser adaptada às necessidades específicas da sua empresa.

Gostaria de agendar uma demonstração para que você possa ver em primeira mão como nossa solução pode transformar a gestão de projetos na sua organização. Por favor, me avise qual dia e horário seriam convenient

### Exercisio

Para aprimorar os resultados obtidos vamos a criar um sistemas multi-agente para conseguir fazer uma análise do problema (reflexão), escrever o email a partir deste plano (escrita) e finalmente avaliar os resultados obtidos (avaliação).

- Step 1: Crie os três agentes (Agente reflexão, Agente escrita, Agente avaliação)
- Step 2: Crie um pipeline (chain) para gerar os resultados
- Step 3: Avalie e intere sobre os resultados obtidos


In [35]:
# TRABALHO GRUPO 02 - ATIVIDADE 03


In [152]:
#agent_reflection > Agente que faz avaliação para escrita do email

from pydantic import BaseModel, Field

# Schema para o Agente de Escrita
class EmailResponse(BaseModel):
    subject: str = Field(..., description="A compelling subject line.")
    body: str = Field(..., description="The full professional email content.")
    tips: str = Field(..., description="Advice for personalization.")
    chain_of_thought: str = Field(..., description="Reasoning on how the strategy was applied.")

# Inicializando a Chain do Escritor
writer_chain = chat.with_structured_output(EmailResponse)

In [153]:
#prompt para a reflexão

from langchain_core.messages import HumanMessage, SystemMessage

# Inicializando o agente com o Schema acima
strategy_chain = chat.with_structured_output(EmailReflection)

# O input é o contexto bruto do problema
contexto_bruto = "Aumento de 15% na infra de nuvem devido à implementação de redundância geográfica."

# Invocando a reflexão estratégica
estrategia_planejada = strategy_chain.invoke([
    SystemMessage(content="Você é o Estrategista de Comunicação."),
    HumanMessage(content=f"Como devo abordar este cliente sobre: {contexto_bruto}?")
])

# Agora você tem um guia para o próximo agente!
print(f"Abordagem: {estrategia_planejada.strategic_approach}")
print(f"O que evitar: {estrategia_planejada.pitfalls_to_avoid}")

Abordagem: Informativo e colaborativo
O que evitar: ['Evitar termos técnicos excessivos que possam confundir o cliente.', 'Não minimizar a importância da redundância geográfica.', 'Não apresentar o aumento como um custo, mas sim como um investimento.', 'Evitar comparações negativas com a infraestrutura atual.']


In [186]:
#agent_email > Agente que faz a escrita do email, com base na reflexão gerada pelo Agent_reflection

from pydantic import BaseModel, Field

class EmailResponse(BaseModel):
    subject: str = Field(..., description="A compelling subject line for the email.")
    body: str = Field(..., description="The full professional email content.")
    tips: str = Field(..., description="Advice on how to personalize this further for the specific client.")
    chain_of_thought: str = Field(..., description="Explanation of how the reflection's strategy was applied to the text.")




In [187]:
#prompt para criação do email - PRIMEIRA ITERAÇÃO (Tentativa de obetenção de avaliação POSITIVA pelo Agent_Evaluation)

# Prompt de Sistema para o Escritor
SYSTEM_PROMPT_WRITER = "Você é um Redator Executivo de elite que transforma estratégias em e-mails impecáveis."

# Construindo o Prompt com a entrada do Agent_reflection
prompt_escrita = f"""
Siga as diretrizes do Estrategista:
- ESTRATÉGIA: {estrategia_planejada.strategic_approach}
- PONTOS DE ATENÇÃO: {estrategia_planejada.pitfalls_to_avoid}

CONTEXTO BRUTO: {contexto_bruto}
ESTILO: {styles[0]}
TOM: {tones[0]}

Escreva o e-mail agora.
"""

# Execução do Agente de Escrita (Email_response)
email_final_gerado = writer_chain.invoke([
    SystemMessage(content=SYSTEM_PROMPT_WRITER),
    HumanMessage(content=prompt_escrita)
])

# --- MOSTRAR O EMAIL GERADO ---
print("\n" + "="*50)
print("📩 E-MAIL GERADO PELO AGENTE DE ESCRITA")
print("="*50)
print(f"ASSUNTO: {email_final_gerado.subject}")
print("-" * 50)
print(f"{email_final_gerado.body}")
print("-" * 50)
print(f"💡 DICA DO ESCRITOR: {email_final_gerado.tips}") #interação do agente com o usuário
print("="*50 + "\n")


📩 E-MAIL GERADO PELO AGENTE DE ESCRITA
ASSUNTO: Atualização sobre a Infraestrutura de Nuvem
--------------------------------------------------
Prezado [Nome do Cliente],

Espero que esta mensagem o encontre bem. 

Gostaria de compartilhar uma atualização importante sobre a nossa infraestrutura de nuvem. Estamos implementando uma melhoria significativa que resultará em um aumento de 15% na capacidade de nossa infraestrutura. Essa expansão é parte de nossa estratégia contínua para garantir a máxima eficiência e segurança em nossos serviços.

A principal razão para esse aumento é a implementação de redundância geográfica. Essa abordagem não apenas fortalece a resiliência de nossos serviços, mas também assegura que seus dados estejam sempre disponíveis, mesmo em situações imprevistas. A redundância geográfica é uma prática recomendada que proporciona uma camada adicional de segurança e confiabilidade, essencial para o sucesso de qualquer operação na nuvem.

Entendemos que qualquer ajuste 

In [194]:
#prompt para criação do email - SEGUNDA ITERAÇÃO (Adaptação para tentativa de obetenção de avaliação 10 pelo Agent_Evaluation)

# --- PROMPT DE SISTEMA: O REDATOR ESTRATÉGICO ---
SYSTEM_PROMPT_WRITER = """
Você é um Redator Executivo especializado em transformar decisões técnicas em valor de negócio.
Sua missão é redigir um e-mail que justifique um investimento de 15% de forma tão clara que o cliente se sinta seguro e bem atendido.

REGRAS PARA APROVAÇÃO (CRÍTICAS DA AUDITORIA):
1. TRADUÇÃO TÉCNICA: Sempre que citar 'Redundância Geográfica', explique-a imediatamente como 'proteção de dados em múltiplos locais para
garantir que sua operação nunca pare'.
2. ANTI-ALARMISMO: Em vez de falar em 'imprevistos' ou 'falhas', use 'disponibilidade contínua' e 'resiliência de dados'.
3. FOCO NO CLIENTE: O aumento de 15% deve ser apresentado como um 'investimento em blindagem operacional'.
4. ESTILO: Mantenha-se Formal e Técnico, mas seja Colaborativo (abra espaço para dúvidas).
"""

# --- PROMPT DE ENTRADA (HUMAN MESSAGE) ---
prompt_escrita = f"""
Com base na reflexão do Estrategista, redija o e-mail final.

[DIRETRIZES DO ESTRATEGISTA]
- ABORDAGEM: {estrategia_planejada.strategic_approach}
- O QUE EVITAR: {estrategia_planejada.pitfalls_to_avoid}

[DADOS DO CONTEXTO]
- FATO: {contexto_bruto}

[INSTRUÇÕES ADICIONAIS PARA NOTA 10]
- Comece agradecendo a parceria.
- Explique que o upgrade para redundância geográfica é um passo estratégico para a segurança dele.
Destaque que hoje não há replicação da base de dados e que isso o deixa em uma condiçao de vulnerabilidade.
- Não utilize o termo redundância geográfica. Ao inves disso, opte por ditribuição em outras regiões para segurança e réplicas dos dados.
- Deixe claro que o valor de 15% é o custo para garantir que o serviço esteja disponível 24/7, independente de fatores externos.
Destaque que o impacto do cusro não é negativo, mas sim um investimento que trará inúmeros benefícios.
- Termine com um convite para uma breve conversa caso ele queira entender os detalhes técnicos da nova arquitetura.

ESTILO: {styles[0]}
TOM: {tones[0]}
"""

# Execução do Agente de Escrita
email_final_gerado = writer_chain.invoke([
    SystemMessage(content=SYSTEM_PROMPT_WRITER),
    HumanMessage(content=prompt_escrita)
])
# --- MOSTRAR O EMAIL GERADO ---
print("\n" + "📧" + "="*48)
print("  E-MAIL GERADO PARA AUDITORIA")
print("="*50)
print(f"ASSUNTO: {email_final_gerado.subject}")
print("-" * 50)
print(f"{email_final_gerado.body}")
print("-" * 50)
print(f"🧠 RACIOCÍNIO DE ESCRITA: {email_final_gerado.chain_of_thought}")
print(f"💡 DICA PARA PERSONALIZAR: {email_final_gerado.tips}")
print("="*50 + "\n")


📧================================================
  E-MAIL GERADO PARA AUDITORIA
ASSUNTO: Proposta de Investimento em Segurança e Disponibilidade de Dados
--------------------------------------------------
Prezado [Nome do Cliente],

Espero que esta mensagem o encontre bem. Gostaria de iniciar agradecendo pela parceria contínua e pela confiança depositada em nossos serviços.

Estou entrando em contato para discutir um passo estratégico que consideramos essencial para a segurança e a continuidade das suas operações. Atualmente, a sua infraestrutura não conta com a replicação da base de dados, o que pode deixá-lo em uma condição de vulnerabilidade. Para mitigar esse risco, propomos um upgrade que envolve a distribuição dos dados em múltiplas regiões. Essa abordagem garantirá que sua operação nunca pare, mesmo diante de eventuais desafios.

O investimento de 15% que estamos sugerindo não deve ser visto como um custo, mas sim como um investimento em blindagem operacional. Essa melhoria pr

In [191]:
#prompt para criação do email - TERCEIRA ITERAÇÃO (Adaptação para tentativa de obetenção de avaliação NEGATIVA pelo Agent_Evaluation)

# 1. Configuração da Chain (Usando a classe EmailResponse definida anteriormente)
email_response_chain = chat.with_structured_output(EmailResponse)

# 2. Dados de entrada
input_data = {
    "context": "Aumento de 15% nos custos de infraestrutura devido à atualização para servidores com proteção contra ataques DDoS e maior velocidade de cache.",
    "style": "Profissional e Direto",
    "tone": "Parceria Estratégica"
}

# 3. Prompt estruturado (Melhorado para garantir a aprovação do auditor)
prompt_final = f"""
Você é um Consultor de Tecnologia focado em Relacionamento.
Escreva um e-mail sobre: {input_data['context']}

DIRETRIZES:
- Estilo: {input_data['style']}
- Tom: {input_data['tone']}
- Data: {get_today_str()}

REQUISITOS:
1. Explique o aumento de 15% como um investimento em segurança e performance (blindagem contra DDoS) e que este é um investimento.
2. Não seja alarmista; foque na continuidade do negócio do cliente. Tranquilize-o quanto aos custos,
ara que fique claro que é um investimento.
3. Não use o termo redundância geográfica, em hipótese alguma.
4. Destaque a importância dessas cópias no que diz repeito à segurança e integridade das informaçoes.
5. Faça a comparação da infraestrutura atual com a nova, mas sem deixar a impressão de que a atual é ineficiente e insegura
6. Explique de forma clara o máximo que puder sobre o tema e proponha uma conversa para alinhar os próximos passos.
"""

# 4. Execução
response = email_response_chain.invoke([
    SystemMessage(content="Você é um redator executivo de elite."),
    HumanMessage(content=prompt_final)
])
# 5. Exibição (Corrigido os nomes das propriedades)
print(f"ASSUNTO: {response.subject}")
print("-" * 30)
print(f"E-MAIL:\n{response.body}") # Corrigido de response_email para body
print("-" * 30)
print(f"💡 DICA: {response.tips}")
print(f"🧠 RACIOCÍNIO: {response.chain_of_thought}")

ASSUNTO: Investimento em Segurança e Performance da Infraestrutura
------------------------------
E-MAIL:
Prezado [Nome do Cliente],

Espero que esta mensagem o encontre bem. 

Gostaria de discutir uma atualização importante que estamos implementando em nossa infraestrutura, que resultará em um aumento de 15% nos custos. Essa mudança é um investimento estratégico em segurança e performance, focando na proteção contra ataques DDoS e na melhoria da velocidade de cache.

Com a crescente ameaça de ataques cibernéticos, a atualização para servidores com proteção robusta contra DDoS é essencial para garantir a continuidade do seu negócio. Essa proteção não apenas resguarda suas operações, mas também assegura a integridade e a segurança das informações que são cruciais para o seu dia a dia. Além disso, a nova infraestrutura proporcionará um desempenho superior, permitindo que suas aplicações funcionem de maneira mais eficiente e rápida.

É importante ressaltar que a infraestrutura atual já é 

In [195]:
#agent_evaluation > Agente que faz avaliação comparativa entre a reflexão e o email gerado
from pydantic import BaseModel, Field

class EvaluationResponse(BaseModel):
    approved: bool = Field(..., description="True if the email is ready to be sent, False if it needs more changes.")
    strengths: list[str] = Field(..., description="List of positive aspects of the email.")
    weaknesses: list[str] = Field(..., description="List of points that could still be improved.")
    final_score: int = Field(..., description="A final strict score from 1 to 10.")
    chain_of_thought: str = Field(..., description="The auditor's reasoning process.")


In [196]:
#prompt para avaliação do email

# 1. Inicializa o Agente de Avaliação (Auditório)
auditor_chain = chat.with_structured_output(EvaluationResponse)

# 2. Definição das Instruções do Auditor
instrucoes_auditor = "Você é um Auditor Final de Qualidade. Sua análise é fria, técnica e rigorosa"


# 3. Preparação do Prompt de Interligação
# Usamos o email_final_gerado (do passo anterior) e a estratégia inicial (estrategia_planejada)
prompt_auditoria = f"""
AUDITE O SEGUINTE E-MAIL GERADO:
---
ASSUNTO: {email_final_gerado.subject}
CORPO: {email_final_gerado.body}
---

ESTRATÉGIA QUE DEVERIA TER SIDO SEGUIDA:
{estrategia_planejada.strategic_approach}

PONTOS QUE DEVERIAM SER EVITADOS:
{estrategia_planejada.pitfalls_to_avoid}

CONTEXTO ORIGINAL: {contexto_bruto}
ESTILO SOLICITADO: {styles[0]}
TOM SOLICITADO: {tones[0]}

DICA: Não seja rígido com jargões técnicos, a menos que eles tornem o texto impossível de ler.
Considere que o receptor é uma pessoa que entende os termos utilizados.
Se o e-mail estiver 'bom o suficiente' para ser enviado sem causar problemas, dê uma nota alta e aprove.

Compare o e-mail gerado com a estratégia proposta e o contexto original.
"""

# 4. Execução do Agente de Auditoria
veredito_final = auditor_chain.invoke([
    SystemMessage(content=instrucoes_auditor),
    HumanMessage(content=prompt_auditoria)
])

# 5. Exibição do Resultado Final Estruturado
print(f"\n" + "="*30)
print(f"🕵️ RELATÓRIO DE AUDITORIA")
print(f"="*30)
print(f"Aprovado para Envio: {'✅ SIM' if veredito_final.approved else '❌ NÃO'}")
print(f"Nota Final: {veredito_final.final_score}/10")
print(f"\n💪 Pontos Fortes: {', '.join(veredito_final.strengths)}")
print(f"⚠️ Pontos de Melhoria: {', '.join(veredito_final.weaknesses)}")
print(f"\n🧠 Raciocínio do Auditor: {veredito_final.chain_of_thought}")
print(f"="*30)


🕵️ RELATÓRIO DE AUDITORIA
Aprovado para Envio: ❌ NÃO
Nota Final: 6/10

💪 Pontos Fortes: Agradecimento pela parceria, o que demonstra um bom relacionamento com o cliente., Clareza na proposta de investimento e seus benefícios., Tom formal e confiante, adequado ao estilo solicitado.
⚠️ Pontos de Melhoria: Uso de termos como 'replicação da base de dados' e 'distribuição dos dados em múltiplas regiões' que podem ser considerados técnicos demais para alguns clientes., A frase 'não deve ser visto como um custo' pode ser interpretada como minimização da importância do investimento., A comparação implícita com a infraestrutura atual pode ser vista como negativa., Falta de uma abordagem mais colaborativa, como envolver o cliente na discussão sobre a proposta.

🧠 Raciocínio do Auditor: O e-mail apresenta uma boa estrutura e um tom formal e confiante, que é adequado para a situação. No entanto, ele peca ao utilizar jargões técnicos que podem confundir o cliente, o que vai contra a estratégia de 

### Extra: Usando ollama LLMs

In [ ]:
#!pip install langchain-community

In [ ]:
from langchain_community.llms import Ollama

# Conecta ao servidor Ollama (local ou remoto)
llm = Ollama(
    model="gemma2:2b",
    base_url="http://ollama:11434"  # troque se estiver remoto
)

# Prompt simples
response = llm.invoke("Olá! Me diga um 'Olá Mundo' bem bonito em português.")

print("Resposta do modelo:")
print(response)

Resposta do modelo:
Olá, mundo! 🌎✨ 

Que a beleza e o encanto da vida fluam por todo o seu caminho. 😉  😊 



**Rascunho Trabalho atv 3**

In [83]:
#rascunho
email_reflection_chain = (
    chat
    .with_structured_output(EmailReflection)
)

reflection_prompt = """
Você é o Revisor de Elite. Sua missão é pegar o e-mail do 'Escritor' e refiná-lo.
Use a técnica de 'Reflexão em Camadas':
1. Camada de Tom: O e-mail deve ser firme, porém acolhedor.
2. Camada de Precisão: Verifique se os dados de infraestrutura e data estão claros.
3. Camada de Concisão: Remova frases que não agregam valor.
"""

response = chat.invoke([HumanMessage(content=reflection_prompt)])
print("DONE")

DONE


In [84]:
#rascunho
email_response_chain = (
    chat
    .with_structured_output(EmailResponse)
)

input_data = {reflection_prompt}

prompt_response = f"""
Com base no contexto: {input_data}
Escreva um e-mail seguindo o estilo {style[0]}, o tom {tone[0]}.
Data de hoje: {get_today_str()}
"""

response = email_response_chain.invoke([HumanMessage(content=prompt_response)])
print("DONE")

NameError: name 'style' is not defined

In [36]:
#agent_response

class EmailResponse(BaseModel):
    response_email: str = Field(..., description="The full email content generated for the customer.")
    tips: str = Field(..., description="Practical advice on the best time to send or how to follow up.")
    chain_of_thought: str = Field(..., description="The reasoning behind choosing specific words and arguments.")
    score: int = Field(..., description="A score from 1 to 10 evaluating how well the email meets the strategic goals.")

In [37]:
email_response_chain = chat.with_structured_output(EmailResponse)

# O input deve conter o contexto do problema e as preferências de estilo
input_data = {
    "context": "Aumento de 15% nos custos de infraestrutura devido à atualização para servidores com proteção contra ataques DDoS e maior velocidade de cache.",
    "style": "Profissional e Direto",
    "tone": "Parceria Estratégica"
}

prompt_final = f"""
Com base no contexto: {input_data['context']}
Escreva um e-mail seguindo o estilo {input_data['style']} e tom {input_data['tone']}.
Data de hoje: {get_today_str()}
"""

response = email_response_chain.invoke([
    HumanMessage(content=prompt_final)
])

print(f"E-mail: {response.response_email}")
#print(f"Raciocínio: {response.chain_of_thought}")

E-mail: Assunto: Atualização sobre Custos de Infraestrutura

Prezados [Nome do Destinatário],

Espero que esta mensagem os encontre bem.

Gostaria de informar que, em decorrência da recente atualização de nossa infraestrutura, que inclui a implementação de servidores com proteção contra ataques DDoS e um aumento significativo na velocidade de cache, observamos um aumento de 15% nos custos operacionais. Essa atualização é fundamental para garantir a segurança e a eficiência de nossos serviços, proporcionando uma experiência ainda melhor para nossos clientes.

Estamos comprometidos em manter a transparência em nossa parceria e acreditamos que esses investimentos são essenciais para fortalecer nossa posição no mercado e garantir a continuidade dos serviços de alta qualidade que oferecemos. 

Agradecemos pela compreensão e estamos à disposição para discutir quaisquer dúvidas ou preocupações que possam surgir. Juntos, podemos continuar a construir uma parceria sólida e estratégica.

Atencio

In [38]:
#agent_reflection
class EmailReflection(BaseModel):
    email_reflection: str = Field(..., description="The final, polished version of the email after reflection.")
    critique: str = Field(..., description="A detailed critique of what was wrong or weak in the initial draft.")
    chain_of_thought: str = Field(..., description="The internal reasoning process: why these specific changes were made.")
    score: int = Field(..., description="A quality score from 1 to 10 for the final version (goal is 9+).")





In [43]:
HumanMessage(content=f"Contexto: {input_data}. Estilo: {styles[0]}.")

# --- PONTE DE INTERLIGAÇÃO ---
# Aqui pegamos o texto que o Agente 1 escreveu para enviar ao Agente de Reflexão
texto_para_revisar = response.response_email

# --- AGENTE 2: REFLEXÃO (Recebe a saída do Agente 1) ---
prompt_reflection = f"""
Você recebeu o seguinte rascunho de e-mail para revisar:
---
{response.response_email}
---
Considere o contexto original: {input_data}
Por favor, realize sua reflexão crítica e entregue a versão final otimizada, destacando que a não execução da atualização pode derrubar o.
"""

# O Agente de Reflexão agora processa o que o Escritor fez
resultado_final = email_reflection_chain.invoke([
    HumanMessage(content=prompt_reflection)
])

# --- RESULTADO FINAL ---
print("E-mail após reflexão:")
print(resultado_final)
print(f"Nota de qualidade: {resultado_final.score}")

NameError: name 'email_reflection_chain' is not defined

In [95]:
#agent_evaluation
from pydantic import BaseModel, Field

class EvaluationResponse(BaseModel):
    approved: bool = Field(..., description="True if the email is ready to be sent, False if it needs more changes.")
    strengths: list[str] = Field(..., description="List of positive aspects of the email.")
    weaknesses: list[str] = Field(..., description="List of points that could still be improved.")
    final_score: int = Field(..., description="A final strict score from 1 to 10.")
    chain_of_thought: str = Field(..., description="The auditor's reasoning process.")



In [102]:
# 1. Inicializa o Agente de Avaliação
auditor_chain = chat.with_structured_output(EvaluationResponse)

# 2. Definição do Agente
from langchain_core.messages import SystemMessage, HumanMessage

# Definindo a mensagem de sistema
instrucoes_do_agente = "Você é um especialista em logística que fala de forma curta e grossa."

# Passando para a chain
resposta = auditor_chain.invoke([
    SystemMessage(content=instrucoes_do_agente), # Aqui define o comportamento
    HumanMessage(content="Onde está meu pacote?") # Aqui é a pergunta do usuário
])

# 3. Pega a saída do Agente de Reflexão (que você já rodou)
email_para_auditar = resultado_final # Pegando o texto final do reflexão

# 4. Prompt de Interligação
prompt_auditoria = f"""
AUDITE O SEGUINTE E-MAIL:
---
{email_para_auditar}
---
CONTEXTO ORIGINAL: {input_data}
ESTILO SOLICITADO: {styles[0]}
TOM SOLICITADO: {tones[0]}

Realize uma auditoria fria e técnica.
"""

# 4. Execução do Último Agente
veredito_final = auditor_chain.invoke([
    SystemMessage(content="Você é o Auditor Final de Qualidade."),
    HumanMessage(content=prompt_auditoria)
])

# 5. Exibição do Resultado
print(f"--- VEREDITO FINAL ---")
print(f"Aprovado para Envio: {'SIM' if veredito_final.approved else 'NÃO'}")
print(f"Nota Final: {veredito_final.final_score}/10")
print(f"Pontos Fortes: {veredito_final.strengths}")
print(f"Raciocínio do Auditor: {veredito_final.chain_of_thought}")

--- VEREDITO FINAL ---
Aprovado para Envio: SIM
Nota Final: 9/10
Pontos Fortes: ['Clareza na comunicação dos custos e atualizações de infraestrutura.', 'Inclusão de riscos associados à não execução da atualização, o que reforça a urgência da mensagem.', 'Tom profissional e direto, adequado para o contexto.', 'Estrutura bem organizada, facilitando a leitura e compreensão.']
Raciocínio do Auditor: O e-mail apresenta uma boa estrutura e clareza na comunicação, abordando de forma eficaz o aumento de custos e a importância da atualização de infraestrutura. A inclusão dos riscos associados à não execução da atualização é um ponto positivo que reforça a necessidade do investimento. No entanto, a linguagem poderia ser um pouco mais assertiva para transmitir maior confiança e urgência. Além disso, uma chamada à ação mais clara poderia melhorar a interação com o destinatário. No geral, o e-mail está bem elaborado e pronto para ser enviado.


In [197]:
# modelo pipelina completo


def pipeline_comunicacao_cliente(contexto_bruto, estilo, tom):
    print(f"🚀 Iniciando Pipeline para: {contexto_bruto}\n")

    # --- PASSO 1: AGENTE REFLECTION (O ESTRATEGISTA) ---
    print("🧠 [1/3] Agent_reflection analisando estratégia...")
    estrategia_planejada = strategy_chain.invoke([
        SystemMessage(content="Você é o Estrategista de Comunicação."),
        HumanMessage(content=f"Como devo abordar este cliente sobre: {contexto_bruto}?")
    ])

    # --- PASSO 2: AGENTE EMAIL (O ESCRITOR) ---
    # Aqui usamos o prompt otimizado para evitar os erros das rodadas anteriores
    print("✍️ [2/3] Agent_email redigindo mensagem...")
    prompt_escrita = f"""
    Com base na estratégia: {estrategia_planejada.strategic_approach}
    Evitando: {estrategia_planejada.pitfalls_to_avoid}
    Contexto: {contexto_bruto}
    Estilo: {estilo} | Tom: {tom}

    Instrução: Traduza termos técnicos para benefícios de segurança e seja proativo.
    """

    email_gerado = email_response_chain.invoke([
        SystemMessage(content="Você é um Redator Executivo que foca em valor e clareza."),
        HumanMessage(content=prompt_escrita)
    ])

    # --- PASSO 3: AGENTE EVALUATION (O AUDITOR AMIGÁVEL) ---
    print("🔍 [3/3] Agent_evaluation auditando qualidade...")
    prompt_auditoria = f"""
    E-MAIL GERADO:
    Assunto: {email_gerado.subject}
    Corpo: {email_gerado.body}

    CONTEXTO ORIGINAL: {contexto_bruto}
    Avalie se está profissional e claro. Seja construtivo e menos rigoroso.
    """

    auditoria = auditor_chain.invoke([
        SystemMessage(content="Você é um Revisor de Sucesso do Cliente que preza pela clareza e parceria."),
        HumanMessage(content=prompt_auditoria)
    ])

    # --- EXIBIÇÃO FINAL DOS RESULTADOS ---
    print("\n" + "="*60)
    print("✨ RESULTADO FINAL DO PIPELINE ✨")
    print("="*60)
    print(f"ESTADO: {'✅ APROVADO' if auditoria.approved else '⚠️ NECESSITA REVISÃO'}")
    print(f"NOTA: {auditoria.final_score}/10")
    print("-" * 60)
    print(f"ASSUNTO: {email_gerado.subject}")
    print(f"\n{email_gerado.body}")
    print("-" * 60)
    print(f"💡 DICA DO ESCRITOR: {email_gerado.tips}")
    print(f"🧐 PONTOS FORTES: {', '.join(auditoria.strengths)}")
    print(f"🧠 RACIOCÍNIO DO AUDITOR: {auditoria.chain_of_thought}")
    print("="*60)

    return {
        "estrategia": estrategia_planejada,
        "email": email_gerado,
        "auditoria": auditoria
    }

# --- EXECUTANDO O PIPELINE ---
resultado = pipeline_comunicacao_cliente(
    contexto_bruto="Aumento de 15% na infra de nuvem devido à implementação de redundância geográfica.",
    estilo="Profissional e Técnico",
    tom="Confiante e Colaborativo"
)

🚀 Iniciando Pipeline para: Aumento de 15% na infra de nuvem devido à implementação de redundância geográfica.

🧠 [1/3] Agent_reflection analisando estratégia...
✍️ [2/3] Agent_email redigindo mensagem...
🔍 [3/3] Agent_evaluation auditando qualidade...

✨ RESULTADO FINAL DO PIPELINE ✨
ESTADO: ✅ APROVADO
NOTA: 8/10
------------------------------------------------------------
ASSUNTO: Atualização Importante: Investimento em Segurança e Confiabilidade da Nuvem

Prezado [Nome do Cliente],

Espero que esteja bem! 

Gostaria de compartilhar uma atualização importante sobre nossa infraestrutura de nuvem. Estamos implementando uma melhoria significativa que resultará em um aumento de 15% em nossa capacidade. Essa mudança é motivada pela introdução de redundância geográfica, uma estratégia que visa aumentar a segurança e a confiabilidade dos seus dados.

A redundância geográfica significa que seus dados serão armazenados em múltiplas localizações, o que não apenas protege contra falhas locais, m